# Exercise 4: Conditions and Actions

In this notebook, a selection of conditions and actions are introduced. These highlight different approaches how to work with annotations.

In [ ]:
%%documentText
The dog barked at the cat.
Dogs, cats and mice are mammals.
Zander and tuna are fishes.
Peter works for Frank.
10€ are less than 100$.

We annotate all sentences that contain a number or a comma using a combination of conditions. Thus, we classify the sentence using two disjunctive properties.

In [ ]:
%displayMode CSV
%csvConfig SentenceWithNumberOrComma
DECLARE Sentence;

// create some simple sentences
(ANY+{-PARTOF(Sentence),-PARTOF(PERIOD)} PERIOD){-> Sentence};

DECLARE SentenceWithNumberOrComma;

Sentence{OR(CONTAINS(NUM),CONTAINS(COMMA))-> SentenceWithNumberOrComma};

Now, we annotate all sentences that contain at least two capitalized words.

In [ ]:
%csvConfig SentenceWith2CapitalizedWords
DECLARE SentenceWith2CapitalizedWords;

// right now we stil need a upper boundary
Sentence{CONTAINS(CW,2,1000)-> SentenceWith2CapitalizedWords};

Now, we annotate all sentences that start with an animal.

In [ ]:
%csvConfig SentenceStartsWithAnimal
DECLARE SentenceStartsWithAnimal;

DECLARE Animal;
WORDLIST AnimalList = 'animals.txt';
MARKFAST(Animal, AnimalList, true);

Sentence{STARTSWITH(Animal)-> SentenceStartsWithAnimal};

Next, we remove all sentences that contain an animal. The UNMARK action is really useful to disentangle rules.

In [ ]:
%csvConfig Sentence

s:Sentence{CONTAINS(Animal)-> UNMARK(s)};

We remove all amounts of money whose amount is less than 50. Almost any boolean expression can be used to represent an implicit condition. Here, we simple define a condition on the feature value.

In [ ]:
%displayMode RUTA_COLORING

DECLARE MoneyAmount(INT amount, STRING currency);
INT value;
(NUM{PARSE(value)} c:SPECIAL){-> CREATE(MoneyAmount, "amount"=value, "currency"=c.ct)};

ma:MoneyAmount{ma.amount<50 -> UNMARK(ma)};

COLOR(MoneyAmount, "lightgreen");

We also remove all amounts of money whose currency is dollar.

In [ ]:
ma:MoneyAmount{ma.currency=="$" -> UNMARK(ma)};

In the next exmaple, we change the offsets of an existing annoation. We expand the "WorksFor"" annotation to the complete document.

In [ ]:
DECLARE Employer, Employee;
"Peter"-> Employee;
"Frank"-> Employer;

DECLARE WorksFor (Employee employee, Employer employer);
(e1:Employee # e2:Employer){-> wf:WorksFor, wf.employee=e1, wf.employer=e2};

// we can use the action SHIFT
//# @WorksFor{-> SHIFT(WorksFor,1,3)} #; 

// or we could do this also using implicit actions:
b:# wf:@WorksFor{-> wf.begin=b.begin, wf.end=e.end} e:#;

COLOR(WorksFor, "pink");

In [ ]:
// reset everything and start anew
%resetCas

In [ ]:
%%documentText
The dog barked at the cat.
Dogs, cats and mice are mammals.
Zander and tuna are fishes.
Peter works for Frank.
10€ are less than 100$.

Now, we trim the sentences by punctuation marks.

In [ ]:
%displayMode CSV
%csvConfig Sentence

DECLARE Sentence;
(ANY+{-PARTOF(Sentence),-PARTOF(PERIOD)} PERIOD){-> Sentence};

Sentence{->TRIM(PERIOD)};

And we split the sentences at the word "are".

In [ ]:
DECLARE Split;
W.ct=="are"{-> Split};

Sentence{-> SPLIT(Split)};

There are many useful actions and conditions. A complete list can be found in the UIMA Ruta documentation.